# Analysis of clients and services per region
Aim: Determine whether homelessness services are sufficient by comparing the number of clients per region with number of services in that region.
1. Obtain table of clients per region
2. For each region, obtain latitudes, longitudes and radius
3. Using data from step 2, perform Nearby Places API search to obtain homelessness services in the region
4. Using jupyter gmaps, create a weighted heatmap of clients per region. Add markers for each service within the region

In [1]:
#Import dependencies
import pandas as pd
from config import gkey
import requests
from pprint import pprint
import json
from math import radians, cos, sin, asin, sqrt
import gmaps
import os
import matplotlib.pyplot as plt

In [2]:
#read table of clients per region
places_df = pd.read_csv("Resources/places.csv")
places_df

,Table of contents,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23
0,"Table CLIENTLOC.1: Clients, by Statistical Are...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,Clients,NaN,NaN,Homeless,NaN,NaN,At Risk,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sex (%),NaN
3,State/territory,Region Code,Region Name,Number,"Rate per 10,000 ERP(b)",NaN,Number,Per cent,NaN,Number,...,15–17,18–24,25–34,35–44,45–54,55–64,65+,NaN,Males,Females
4,NSW,101,Capital Region,"3,430",149.2,NaN,"1,365",41.9,NaN,"1,891",...,7.4,16.0,14.8,15.1,11.2,5.7,3.0,NaN,46.4,53.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,(c) Includes clients from 'Other territories' ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,Notes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99,1. Data presented have not been adjusted fo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,2. Clients are assigned to a region based o...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#Clean dataframe
#Create copy of dataframe
clean_places_df = places_df.copy()
#Set column headers
clean_places_df.columns = clean_places_df.iloc[3]
#Only obtain data from WA
clean_places_df = clean_places_df.loc[clean_places_df["State/territory"] == "WA"]
#Remove unwanted columns
clean_places_df = clean_places_df.iloc[:,range(0,4)]
#Reindex dataframe
clean_places_df = clean_places_df.reset_index(drop = True)
#Convert datatypes to floats
clean_places_df["Number"] = clean_places_df["Number"].str.replace(",","")
clean_places_df = clean_places_df.astype({"Region Code": "float"}, {"Number":"float"})
clean_places_df

3,State/territory,Region Code,Region Name,Number
0,WA,501.0,Bunbury,1598
1,WA,502.0,Mandurah,1266
2,WA,503.0,Perth - Inner,1527
3,WA,504.0,Perth - North East,1244
4,WA,505.0,Perth - North West,1772
5,WA,506.0,Perth - South East,2607
6,WA,507.0,Perth - South West,3083
7,WA,509.0,Western Australia - Wheat Belt,1308
8,WA,510.0,Western Australia - Outback (North),5712
9,WA,511.0,Western Australia - Outback (South),2184


In [4]:
#Add latitude and longitude columns to dataframe
clean_places_df["Mid Latitude"] = " "
clean_places_df["Mid Longitude"] = " "
clean_places_df["Radius (m)"] = " "
clean_places_df

3,State/territory,Region Code,Region Name,Number,Mid Latitude,Mid Longitude,Radius (m)
0,WA,501.0,Bunbury,1598,,,
1,WA,502.0,Mandurah,1266,,,
2,WA,503.0,Perth - Inner,1527,,,
3,WA,504.0,Perth - North East,1244,,,
4,WA,505.0,Perth - North West,1772,,,
5,WA,506.0,Perth - South East,2607,,,
6,WA,507.0,Perth - South West,3083,,,
7,WA,509.0,Western Australia - Wheat Belt,1308,,,
8,WA,510.0,Western Australia - Outback (North),5712,,,
9,WA,511.0,Western Australia - Outback (South),2184,,,


# Issues:
Regions are defined by the Australian Statistical Geography Standard (ASGS), which is set by the Australian Beaureau of Statistics (ABS). 
Coordinates for each region could not be found from the ASGS.
Found a table detailing postcodes, latitudes and longitudes for each region. 
This will be used to obtain a mid-point latitude and longitude for each region, and to calculate the radius of each region.

In [5]:
#Read table with postcodes, latitudes and longitudes for each region code
postcodes_df = pd.read_csv("Resources/australian_postcodes.csv")
postcodes_df

,id,postcode,locality,state,long,lat,dc,type,status,sa3,...,SA3_NAME_2016,SA4_CODE_2016,SA4_NAME_2016,RA_2011,RA_2016,MMM_2015,MMM_2019,ced,altitude,chargezone
0,230,200,ANU,ACT,149.119000,-35.277700,NaN,NaN,NaN,NaN,...,North Canberra,801.0,Australian Capital Territory,1.0,1.0,1.0,1.0,NaN,NaN,N2
1,21820,200,Australian National University,ACT,149.118900,-35.277700,NaN,NaN,Added 19-Jan-2020,NaN,...,North Canberra,801.0,Australian Capital Territory,1.0,1.0,1.0,1.0,NaN,NaN,N2
2,232,800,DARWIN,NT,130.836680,-12.458684,NaN,NaN,Updated 6-Feb-2020,70101.0,...,Darwin City,701.0,Darwin,3.0,3.0,2.0,2.0,NaN,NaN,NT1
3,233,801,DARWIN,NT,130.836680,-12.458684,NaN,NaN,Updated 25-Mar-2020 SA3,70101.0,...,Darwin City,701.0,Darwin,3.0,3.0,2.0,2.0,NaN,NaN,NT1
4,234,804,PARAP,NT,130.873315,-12.428017,NaN,NaN,Updated 25-Mar-2020 SA3,70102.0,...,Darwin City,701.0,Darwin,3.0,3.0,2.0,2.0,NaN,NaN,NT1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18438,11186,9013,BRISBANE,QLD,152.823141,-27.603479,CITY DC - BRISBANE,LVR,Updated 25-Mar-2020 SA3,30504.0,...,Brisbane Inner,305.0,Brisbane Inner City,1.0,1.0,1.0,1.0,NaN,44.349792,Q1
18439,11187,9015,BRISBANE,QLD,152.823141,-27.603479,CITY DC - BRISBANE,LVR,Updated 25-Mar-2020 SA3,30504.0,...,Brisbane Inner,305.0,Brisbane Inner City,1.0,1.0,1.0,1.0,NaN,44.349792,Q1
18440,11196,9464,NORTHGATE MC,QLD,153.074982,-27.397055,NaN,NaN,Updated 25-Mar-2020 SA3,30203.0,...,Nundah,302.0,Brisbane - North,1.0,1.0,1.0,1.0,NaN,NaN,Q1
18441,11197,9726,GOLD COAST MC,QLD,153.412197,-28.008783,NaN,NaN,Updated 25-Mar-2020 SA3,30910.0,...,Surfers Paradise,309.0,Gold Coast,1.0,1.0,1.0,1.0,Moncrieff,NaN,Q1


In [6]:
#Clean the dataframe to obtain all regions within WA.
wa_postcodes = postcodes_df.copy()
wa_postcodes = wa_postcodes[["long", "lat", "SA4_CODE_2016", "postcode", "locality"]]
wa_postcodes = wa_postcodes.loc[(wa_postcodes["SA4_CODE_2016"]>=500)
                                & (wa_postcodes["SA4_CODE_2016"]< 600)]
wa_postcodes

,long,lat,SA4_CODE_2016,postcode,locality
331,131.298809,-21.949513,510.0,872,GIBSON DESERT NORTH
332,125.984184,-24.947259,511.0,872,GIBSON DESERT SOUTH
352,126.954107,-23.280767,510.0,872,KIWIRRKURRA
355,129.836553,-22.567705,510.0,872,LAKE MACKAY
368,128.115080,-25.270732,511.0,872,NGAANYATJARRA-GILES
...,...,...,...,...,...
17591,116.009575,-32.039597,506.0,6989,MADDINGTON
17592,116.008461,-32.060407,506.0,6990,GOSNELLS
17593,116.095955,-32.097880,506.0,6991,KELMSCOTT
17594,115.920395,-31.964503,506.0,6992,ARMADALE


In [7]:
#Loop through region codes, obtaining average latitude and longitude, and radius of the region
#Assume each region is a circle
for index, row in clean_places_df.iterrows():
    region_code = row[1]
    max_lat = wa_postcodes["lat"].loc[wa_postcodes["SA4_CODE_2016"] == region_code].max()
    min_lat = wa_postcodes["lat"].loc[wa_postcodes["SA4_CODE_2016"] == region_code].min()
    max_long = wa_postcodes["long"].loc[wa_postcodes["SA4_CODE_2016"] == region_code].max()
    min_long = wa_postcodes["long"].loc[wa_postcodes["SA4_CODE_2016"] == region_code].min()
    mid_lat = wa_postcodes["lat"].loc[wa_postcodes["SA4_CODE_2016"] == region_code].mean()
    mid_long = wa_postcodes["long"].loc[wa_postcodes["SA4_CODE_2016"] == region_code].mean()
    clean_places_df.loc[index, "Mid Latitude"] = mid_lat
    clean_places_df.loc[index, "Mid Longitude"] = mid_long
    max_lat, min_lat, max_long, min_long = map(radians,[max_lat, min_lat, max_long, min_long])
    dlat = max_lat - min_lat
    dlong = max_long - min_long
    a = sin(dlat/2)**2 + cos(min_lat) * cos(max_lat) * sin(dlong/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Determines return value units.
    radius = c * r *1000
    clean_places_df.loc[index, "Radius (m)"] = radius
clean_places_df

3,State/territory,Region Code,Region Name,Number,Mid Latitude,Mid Longitude,Radius (m)
0,WA,501.0,Bunbury,1598,-33.774791,115.78802,259975.615431
1,WA,502.0,Mandurah,1266,-32.584286,115.772537,34038.225809
2,WA,503.0,Perth - Inner,1527,-31.98777,115.803919,215989.507901
3,WA,504.0,Perth - North East,1244,-31.846262,116.031049,69896.483766
4,WA,505.0,Perth - North West,1772,-31.7989,115.794336,121544.6306
5,WA,506.0,Perth - South East,2607,-32.082549,115.987136,203313.748938
6,WA,507.0,Perth - South West,3083,-32.152059,115.806404,67210.015891
7,WA,509.0,Western Australia - Wheat Belt,1308,-32.444299,117.39199,765064.046357
8,WA,510.0,Western Australia - Outback (North),5712,-19.947728,120.793778,1944829.77484
9,WA,511.0,Western Australia - Outback (South),2184,-28.910203,118.024281,2255197.173029


In [38]:
#Save to dataframe
clean_places_df.to_csv("Resources/clients_per_region.csv")

# Issues:
Initially calculated a midpoint of latitudes and longitudes, however discovered that the midpoints were not accurate. Therefore used mean latitude and longitude as a more accurate mid-point.

In [9]:
#Perform search of homeless shelters using Nearby Places Google API
service_name = []
service_latitude = []
service_longitude = []


for index, rows in clean_places_df.iterrows():
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "location":(f'{rows["Mid Latitude"]},{rows["Mid Longitude"]}'),
        "keyword":"homeless service",
        "radius":{rows["Radius (m)"]},
        "key":{gkey}
    }

    response = requests.get(url, params = params).json()
    
    if response['status'] == "ZERO_RESULTS":
        print(f"No data could be retrieved for region: {rows['Region Name']}")
        
    else:   
        print(f"Retrieving results for region: {rows['Region Name']}")
        for record in response["results"]:
            service_name.append(record["name"])
            service_latitude.append(record["geometry"]["location"]["lat"])
            service_longitude.append(record["geometry"]["location"]["lng"])

No data could be retrieved for region: Bunbury
Retrieving results for region: Mandurah
Retrieving results for region: Perth - Inner
Retrieving results for region: Perth - North East
Retrieving results for region: Perth - North West
Retrieving results for region: Perth - South East
Retrieving results for region: Perth - South West
No data could be retrieved for region: Western Australia - Wheat Belt
No data could be retrieved for region: Western Australia - Outback (North)
No data could be retrieved for region: Western Australia - Outback (South)


In [10]:
#Create dataframe of shelter names, latitudes and longitudes
services_df = pd.DataFrame({
    "Services Names":service_name,
    "Services Latitudes":service_latitude,
    "Services Longitudes":service_longitude
})
services_df

#Saved dataframe into a csv file
services_df.to_csv("Resources/services.csv", header = True)

In [11]:
#Read the dataframe for use in jupyter gmaps
services = pd.read_csv("Resources/services.csv", index_col = [0])
services

,Services Names,Services Latitudes,Services Longitudes
0,Bradley Holloway,-32.366232,115.754568
1,Westaus Crisis & Welfare Services,-32.537846,115.742355
2,The C.R.E.W - Helping those in need,-32.279263,115.737228
3,Peel Passages Resource Centre,-32.534491,115.725251
4,Ruah Community Services,-32.287597,115.740544
...,...,...,...
109,Homeless Healthcare,-31.943282,115.874149
110,Mission Australia Drug and Alcohol Youth Service,-31.953638,115.870080
111,Ruah Community Services,-32.057766,115.748378
112,Ruah Centre,-31.945490,115.854881


# Issues:
Nearby Places API did not obtain homelessness services in remote locations. A "Zero Results" error arose.
Could not find a list of regional homelessness services. 
Therefore, created a list of regional homelessness services using Google Maps.

In [18]:
#Manually compiled list of homelessness services in regional WA
regional_services = pd.read_csv("Resources/homeless_services_regional.csv")
regional_services.head()

,homeless_service_name,address
0,Old Theda,"Theda Station Airport, Drysdale River WA 6740"
1,Garl Garl Walbu Sobering Up Shelter,"23 Stanley Street, Derby WA 6728"
2,Ngnowar-Aerwah Aboriginal Corporation,"471 Great Northern Highway, Wyndham WA 6740"
3,Centacare Kimberley,"23 Robinson Street, Broome WA 6725"
4,Australian Red Cross,"2/38 Frederick Street, Broome WA 6725"


In [19]:
#Obtain latitude and longitude of each service.
regional_services["latitude"] = " "
regional_services["longitude"] = " "
regional_services.head()

,homeless_service_name,address,latitude,longitude
0,Old Theda,"Theda Station Airport, Drysdale River WA 6740",,
1,Garl Garl Walbu Sobering Up Shelter,"23 Stanley Street, Derby WA 6728",,
2,Ngnowar-Aerwah Aboriginal Corporation,"471 Great Northern Highway, Wyndham WA 6740",,
3,Centacare Kimberley,"23 Robinson Street, Broome WA 6725",,
4,Australian Red Cross,"2/38 Frederick Street, Broome WA 6725",,


In [20]:
#Use Geocoding API to obtain coordinates for each service
for index, rows in regional_services.iterrows():
    
    url = "https://maps.googleapis.com/maps/api/geocode/json?"

    params = {
        "address": (f"{rows['address']}"),
        "key":{gkey}
    }

    response = requests.get(url,params = params).json()
    try:
        lat = response["results"][0]["geometry"]["location"]["lat"]
        long = response["results"][0]["geometry"]["location"]["lng"]
        regional_services.loc[index, "latitude"] = lat
        regional_services.loc[index, "longitude"] = long
        print(f"Retrieving coordinates of service: {rows['homeless_service_name']}")
    except:
        print(f"Could not retrieve coordinates of service: {rows['homeless_service_name']}")

Retrieving coordinates of service: Old Theda
Retrieving coordinates of service: Garl Garl Walbu Sobering Up Shelter
Retrieving coordinates of service: Ngnowar-Aerwah Aboriginal Corporation
Retrieving coordinates of service: Centacare Kimberley
Retrieving coordinates of service: Australian Red Cross
Retrieving coordinates of service: Mens Outreach Service Inc
Retrieving coordinates of service: Mission Australia
Retrieving coordinates of service: Save the Children
Retrieving coordinates of service: Mission Australian Newman
Retrieving coordinates of service: East Pilbara Independence Support
Retrieving coordinates of service: Mission Australia Tom Price
Retrieving coordinates of service: Mission Australia South Hedland
Retrieving coordinates of service: Port Hedland Sobering-Up Centre Inc.
Retrieving coordinates of service: MacKillop Family Services Port Hedland
Retrieving coordinates of service: Youth Involvement Council
Retrieving coordinates of service: Karratha Youth Housing Project


In [39]:
regional_services.head()

,Services Names,address,Services Latitudes,Services Longitudes
0,Old Theda,"Theda Station Airport, Drysdale River WA 6740",-14.790961,126.497624
1,Garl Garl Walbu Sobering Up Shelter,"23 Stanley Street, Derby WA 6728",-17.307164,123.639825
2,Ngnowar-Aerwah Aboriginal Corporation,"471 Great Northern Highway, Wyndham WA 6740",-15.487314,128.122563
3,Centacare Kimberley,"23 Robinson Street, Broome WA 6725",-17.95989,122.237839
4,Australian Red Cross,"2/38 Frederick Street, Broome WA 6725",-17.95497,122.236091


In [35]:
#Append information to services dataframe
#Rename columns from regional services dataframe to match services dataframe
regional_services = regional_services.rename(columns = {"homeless_service_name":"Services Names",
                                                        "latitude":"Services Latitudes",
                                                        "longitude":"Services Longitudes"
                                                       })
#Concatenate dataframes
joined_services = pd.concat([services, regional_services], ignore_index = True)
joined_services = joined_services[["Services Names", "Services Latitudes", "Services Longitudes"]]
joined_services

,Services Names,Services Latitudes,Services Longitudes
0,Bradley Holloway,-32.366232,115.754568
1,Westaus Crisis & Welfare Services,-32.537846,115.742355
2,The C.R.E.W - Helping those in need,-32.279263,115.737228
3,Peel Passages Resource Centre,-32.534491,115.725251
4,Ruah Community Services,-32.287597,115.740544
...,...,...,...
158,Centrecare,-33.346239,115.640367
159,Anglicare WA,-33.652319,115.345555
160,River Angels,-33.987387,114.993655
161,Just Home Margaret River Inc,-33.950489,115.071111


In [28]:
# Using the template add the shelter marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Services Names}</dd>
</dl>
"""
# Store the DataFrame Row
services_info = [info_box_template.format(**row) for index, row in joined_services.iterrows()]
locations = joined_services[["Services Latitudes", "Services Longitudes"]]

In [34]:
#Configure API key
gmaps.configure(api_key = gkey)

#Set marker locations
marker_locations = joined_services[["Services Latitudes", "Services Longitudes"]]
locations = clean_places_df[["Mid Latitude", "Mid Longitude"]]
client_numbers = clean_places_df["Number"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

#Add marker layer
markers = gmaps.marker_layer(marker_locations, info_box_content = services_info)
fig.add_layer(markers)

#Add heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=client_numbers, 
                                 dissipating=False, max_intensity=4000,
                                 point_radius=1)
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

# Observations
1. Perth has the greatest number of clients (10,223), followed by the WA Outback (North) (5,712).
2. Based on the visual, Perth has a lot more services for the number of clients, in comparison to WA Outback (North). This indicates that WA Outback (North) have a smaller capacity to service their clients.